### Get the csv path

In [1]:
import pandas as pd 
import os

csv_path = os.environ["CSV_FILE_DIR"]
if csv_path.endswith("/"):
     csv_path = csv_path + "data.csv"   
else:
    csv_path = csv_path + "/data.csv" 

In [2]:
print("running bash command to check if file is there")
! ls -la $csv_path

running bash command to check if file is there
-rw-r--r-- 1 root root 137245 Feb  8 13:50 /home/data.csv


### Spark Code

In [3]:
# Import Spark Library
from pyspark.sql import SparkSession, functions as sqlfun

** Note: Depending on how you have configured Spark in your spark config files and depending on how you defined spark w.r.t jupyter, the following line of code might not work and/or be needed explicitly for the other code to run. **

In [4]:
## Create Spark Session
spark = SparkSession.builder.master("local").appName("Read CSV").getOrCreate()

In [5]:
## Read the csv file
try:
    csv_dataframe = spark.read.csv(csv_path,header=True)
    print("CSV read into Spark Data Frame")
except:
    print("ERROR: Unable to load file")     

CSV read into Spark Data Frame


In [6]:
csv_dataframe.show()

+---+---------+-----+---------+
|age| birthday|  zip|     city|
+---+---------+-----+---------+
| 24|3/22/1982|70052| Adoamalu|
| 45|4/15/1991|77601|Somepufad|
| 58|1/29/1956|99244| Muhejcab|
| 23| 1/9/1967|11846| Bitheuli|
| 18|5/23/1963|86903| Ruziffor|
| 35| 8/7/1997|27040|Ejapovodo|
| 31|12/3/1996|13321|Berapizam|
| 37| 5/2/1991|81671|  Saggeub|
| 26|9/24/1959|95581| Cigkesah|
| 45|3/25/1980|77594|   Koveus|
| 35|6/30/1988|91047|   Nuuzte|
| 40|4/14/1968|46524|   Uczaja|
| 39|5/13/1975|19801|Githiznob|
| 23|9/25/1989|82620|Wewkabkim|
| 18|3/17/1964|79747| Vezonave|
| 48|2/17/1989|06611| Nurgazbe|
| 33|4/11/1971|93393| Iwtukmad|
| 28|1/31/1976|18158|  Oweziri|
| 51| 4/8/2000|70791| Atozibin|
| 36| 8/3/1976|02488| Toruzreb|
+---+---------+-----+---------+
only showing top 20 rows



In [7]:
csv_dataframe.printSchema()

root
 |-- age: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- city: string (nullable = true)



In [8]:
# define a new dataframe with a better schema 
#root
# |-- age: integer (nullable = true)
# |-- birthday: date (nullable = true)
# |-- zip: string (nullable = true)
# |-- city: string (nullable = true)

csv_dataframe_new_schema = csv_dataframe.select(sqlfun.col("age").cast("integer"),
                                                sqlfun.from_unixtime(sqlfun.unix_timestamp('birthday', 'MM/dd/yyy')).cast('date').alias("birthday"),
                                                "zip",
                                                "city")

In [9]:
csv_dataframe_new_schema.printSchema()

root
 |-- age: integer (nullable = true)
 |-- birthday: date (nullable = true)
 |-- zip: string (nullable = true)
 |-- city: string (nullable = true)



In [10]:
csv_dataframe_new_schema.show()

+---+----------+-----+---------+
|age|  birthday|  zip|     city|
+---+----------+-----+---------+
| 24|1982-03-22|70052| Adoamalu|
| 45|1991-04-15|77601|Somepufad|
| 58|1956-01-29|99244| Muhejcab|
| 23|1967-01-09|11846| Bitheuli|
| 18|1963-05-23|86903| Ruziffor|
| 35|1997-08-07|27040|Ejapovodo|
| 31|1996-12-03|13321|Berapizam|
| 37|1991-05-02|81671|  Saggeub|
| 26|1959-09-24|95581| Cigkesah|
| 45|1980-03-25|77594|   Koveus|
| 35|1988-06-30|91047|   Nuuzte|
| 40|1968-04-14|46524|   Uczaja|
| 39|1975-05-13|19801|Githiznob|
| 23|1989-09-25|82620|Wewkabkim|
| 18|1964-03-17|79747| Vezonave|
| 48|1989-02-17|06611| Nurgazbe|
| 33|1971-04-11|93393| Iwtukmad|
| 28|1976-01-31|18158|  Oweziri|
| 51|2000-04-08|70791| Atozibin|
| 36|1976-08-03|02488| Toruzreb|
+---+----------+-----+---------+
only showing top 20 rows



#### Columns in the dataset

In [11]:
csv_dataframe_new_schema.columns

['age', 'birthday', 'zip', 'city']

#### datatype of the columns

In [12]:
csv_dataframe_new_schema.dtypes

[('age', 'int'), ('birthday', 'date'), ('zip', 'string'), ('city', 'string')]

#### shape of dataset i.e number of rows and columns

In [13]:
csv_dataframe_new_schema.count() #number of rows

5000

#### number of columns

In [14]:
len(csv_dataframe_new_schema.columns)

4

#### performing some sorting of data

In [15]:
from pyspark.sql.functions import desc

In [16]:
csv_dataframe_new_schema.sort(desc("age")).show() 

+---+----------+-----+---------+
|age|  birthday|  zip|     city|
+---+----------+-----+---------+
| 65|1959-12-24|20590| Ogwigdov|
| 65|1975-01-31|13199| Etuheoju|
| 65|1977-07-03|99900|Dopijooko|
| 65|1961-08-11|50253|  Pilokge|
| 65|1957-03-26|37751|  Noubido|
| 65|1970-05-12|12765|  Zeehfek|
| 65|1996-07-09|69750|  Mesopni|
| 65|1985-09-30|34277| Gadwopod|
| 65|1994-11-11|44892| Bacekrer|
| 65|1996-04-07|08338|  Pihozok|
| 65|1991-09-27|30943|   Hohazu|
| 65|1981-12-06|74061|Erazutkiz|
| 65|1962-10-04|49070|Nazwezdak|
| 65|1954-02-13|44773|Toggimjiz|
| 65|1996-02-03|98919|   Rizoto|
| 65|1956-06-23|41755|  Tovezab|
| 65|1962-09-10|01472|Hegujibod|
| 65|1991-11-04|42222| Tehporam|
| 65|1970-11-22|39680| Mituwfog|
| 65|1958-07-29|57335|  Visakin|
+---+----------+-----+---------+
only showing top 20 rows



### finding the number of cities that falls within a certain age

In [17]:
csv_dataframe_new_schema.groupby(['age']).agg({'city' : 'count'}).sort(desc("age")).show(100)

+---+-----------+
|age|count(city)|
+---+-----------+
| 65|         91|
| 64|        103|
| 63|        111|
| 62|        101|
| 61|         88|
| 60|        105|
| 59|        115|
| 58|         89|
| 57|        122|
| 56|         96|
| 55|        113|
| 54|         83|
| 53|        113|
| 52|        104|
| 51|        100|
| 50|         90|
| 49|        100|
| 48|         97|
| 47|        104|
| 46|         97|
| 45|        116|
| 44|         96|
| 43|        105|
| 42|        104|
| 41|         98|
| 40|        112|
| 39|        134|
| 38|        105|
| 37|        101|
| 36|        125|
| 35|        126|
| 34|         93|
| 33|         96|
| 32|        119|
| 31|         92|
| 30|         94|
| 29|        123|
| 28|         97|
| 27|         96|
| 26|        108|
| 25|         99|
| 24|        108|
| 23|         86|
| 22|        116|
| 21|        105|
| 20|         96|
| 19|        128|
| 18|        100|
+---+-----------+

